In [141]:
import pandas as pd
import numpy as np
import multiprocessing as mp
import psutil
import random
import datetime as datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [142]:
# rand_sample_csv is a randomized subset (1% the size) of the sample_csv which is ~400k instances
df = pd.read_csv('/Users/samkahr/amli/ad_optimization/notebooks/rand_sample_csv.csv')


In [143]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,click,hour,C1,banner_pos,site_id,site_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,0,0,236,13120299559997056165,0,14102100,1005,0,543a539e,c7ca3108,...,1,0,20362,320,50,2333,0,39,-1,157
1,1,1,259,13447361190641805430,0,14102100,1005,1,17caea14,0dde25ec,...,1,0,19950,320,50,1800,3,167,100075,23
2,2,2,357,14758321504714974000,0,14102100,1005,0,85f751fd,c4e18dd6,...,1,0,19743,320,50,2264,3,427,100000,61
3,3,3,455,16246356889796608406,0,14102100,1005,0,5b08c53b,7687a86e,...,1,0,17654,300,250,1994,2,39,100084,33
4,4,4,540,17569568361412548369,0,14102100,1005,0,543a539e,c7ca3108,...,1,0,20362,320,50,2333,0,39,-1,157


In [144]:
df.describe()


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,click,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,4184.000000,4184.000000,4184.000000,4.184000e+03,4184.000000,4.184000e+03,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000,4184.000000
mean,2091.500000,2091.500000,202351.664675,9.113168e+18,0.170172,1.410256e+07,1004.977533,0.283222,1.012189,0.331501,18728.379302,318.803059,60.868069,2100.065488,1.412285,229.095841,53381.884082,84.231836
std,1207.961092,1207.961092,116384.632429,5.389524e+18,0.375829,2.963423e+02,1.032567,0.489278,0.488054,0.856303,5058.361185,22.743031,48.729395,621.145767,1.324642,358.883751,49952.289351,70.673532
min,0.000000,0.000000,236.000000,1.513211e+15,0.000000,1.410210e+07,1002.000000,0.000000,0.000000,0.000000,375.000000,216.000000,36.000000,112.000000,0.000000,33.000000,-1.000000,13.000000
25%,1045.750000,1045.750000,99768.500000,4.449284e+18,0.000000,1.410230e+07,1005.000000,0.000000,1.000000,0.000000,16687.000000,320.000000,50.000000,1800.000000,0.000000,35.000000,-1.000000,23.000000
50%,2091.500000,2091.500000,204607.500000,9.042769e+18,0.000000,1.410260e+07,1005.000000,0.000000,1.000000,0.000000,20312.000000,320.000000,50.000000,2314.000000,2.000000,39.000000,100053.500000,61.000000
75%,3137.250000,3137.250000,301481.250000,1.381808e+19,0.000000,1.410281e+07,1005.000000,1.000000,1.000000,0.000000,21893.000000,320.000000,50.000000,2526.000000,3.000000,169.000000,100084.750000,111.000000
max,4183.000000,4183.000000,404396.000000,1.844673e+19,1.000000,1.410302e+07,1012.000000,7.000000,5.000000,5.000000,24041.000000,728.000000,480.000000,2756.000000,3.000000,1839.000000,100248.000000,255.000000


In [145]:
#dropping unnamed columns
#df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

df.head()


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,click,hour,C1,banner_pos,site_id,site_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,0,0,236,13120299559997056165,0,14102100,1005,0,543a539e,c7ca3108,...,1,0,20362,320,50,2333,0,39,-1,157
1,1,1,259,13447361190641805430,0,14102100,1005,1,17caea14,0dde25ec,...,1,0,19950,320,50,1800,3,167,100075,23
2,2,2,357,14758321504714974000,0,14102100,1005,0,85f751fd,c4e18dd6,...,1,0,19743,320,50,2264,3,427,100000,61
3,3,3,455,16246356889796608406,0,14102100,1005,0,5b08c53b,7687a86e,...,1,0,17654,300,250,1994,2,39,100084,33
4,4,4,540,17569568361412548369,0,14102100,1005,0,543a539e,c7ca3108,...,1,0,20362,320,50,2333,0,39,-1,157


In [146]:
#finding out how many unique values are in each col

for col in df.columns.values:
    print(str(col)+ ": " +  "{}".format(len(df[col].unique())) )

Unnamed: 0: 4184
Unnamed: 0.1: 4184
Unnamed: 0.1.1: 4184
id: 4184
click: 2
hour: 240
C1: 5
banner_pos: 5
site_id: 354
site_domain: 292
site_category: 14
app_id: 244
app_domain: 32
app_category: 14
device_id: 660
device_ip: 3908
device_model: 871
device_type: 4
device_conn_type: 4
C14: 675
C15: 4
C16: 5
C17: 279
C18: 4
C19: 59
C20: 105
C21: 54


In [147]:
#finding out 
df.corr()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,click,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
Unnamed: 0,1.000000,1.000000,0.999890,0.006117,-0.010643,0.995098,0.025705,0.032567,0.024377,0.060443,0.199504,0.023677,0.013901,0.193322,-0.008764,0.053409,0.023395,0.072081
Unnamed: 0.1,1.000000,1.000000,0.999890,0.006117,-0.010643,0.995098,0.025705,0.032567,0.024377,0.060443,0.199504,0.023677,0.013901,0.193322,-0.008764,0.053409,0.023395,0.072081
Unnamed: 0.1.1,0.999890,0.999890,1.000000,0.006268,-0.010592,0.995386,0.025975,0.032781,0.024562,0.060867,0.198020,0.023707,0.013953,0.191661,-0.006952,0.053823,0.023665,0.069815
id,0.006117,0.006117,0.006268,1.000000,0.005248,0.009534,0.010710,0.021110,0.007537,-0.010223,-0.016713,0.024989,-0.038586,-0.015281,-0.003459,-0.013934,-0.012295,-0.002716
click,-0.010643,-0.010643,-0.010592,0.005248,1.000000,-0.008430,-0.016635,0.023851,-0.026951,-0.095847,-0.066833,-0.033668,0.100068,-0.062561,0.011742,0.014138,-0.064616,-0.064840
hour,0.995098,0.995098,0.995386,0.009534,-0.008430,1.000000,0.024804,0.035999,0.023275,0.060760,0.204734,0.026040,0.013443,0.197093,-0.005527,0.054076,0.022392,0.073382
C1,0.025705,0.025705,0.025975,0.010710,-0.016635,0.024804,1.000000,0.252980,0.869132,0.194173,0.062748,0.128507,0.047501,0.070553,-0.036397,-0.001061,-0.035674,0.039389
banner_pos,0.032567,0.032567,0.032781,0.021110,0.023851,0.035999,0.252980,1.000000,0.293886,-0.103751,-0.019084,0.033995,-0.038550,-0.037367,0.119304,0.116407,0.064685,-0.112751
device_type,0.024377,0.024377,0.024562,0.007537,-0.026951,0.023275,0.869132,0.293886,1.000000,0.222572,0.048779,0.196272,0.046116,0.052380,-0.039946,0.000395,-0.045338,0.036166
device_conn_type,0.060443,0.060443,0.060867,-0.010223,-0.095847,0.060760,0.194173,-0.103751,0.222572,1.000000,0.061886,0.068597,-0.022470,0.075564,-0.056449,0.002714,0.105090,0.062582


In [148]:
# check for missing values
df.isnull().sum()

Unnamed: 0          0
Unnamed: 0.1        0
Unnamed: 0.1.1      0
id                  0
click               0
hour                0
C1                  0
banner_pos          0
site_id             0
site_domain         0
site_category       0
app_id              0
app_domain          0
app_category        0
device_id           0
device_ip           0
device_model        0
device_type         0
device_conn_type    0
C14                 0
C15                 0
C16                 0
C17                 0
C18                 0
C19                 0
C20                 0
C21                 0
dtype: int64

In [149]:
# separate the date and time
parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')
df['new_hour'] = df.hour.astype(str).apply(parse_date)
df['new_hour']

0      2014-10-21 00:00:00
1      2014-10-21 00:00:00
2      2014-10-21 00:00:00
3      2014-10-21 00:00:00
4      2014-10-21 00:00:00
5      2014-10-21 00:00:00
6      2014-10-21 00:00:00
7      2014-10-21 00:00:00
8      2014-10-21 00:00:00
9      2014-10-21 00:00:00
10     2014-10-21 00:00:00
11     2014-10-21 01:00:00
12     2014-10-21 01:00:00
13     2014-10-21 01:00:00
14     2014-10-21 01:00:00
15     2014-10-21 01:00:00
16     2014-10-21 01:00:00
17     2014-10-21 01:00:00
18     2014-10-21 01:00:00
19     2014-10-21 01:00:00
20     2014-10-21 01:00:00
21     2014-10-21 01:00:00
22     2014-10-21 01:00:00
23     2014-10-21 01:00:00
24     2014-10-21 01:00:00
25     2014-10-21 01:00:00
26     2014-10-21 01:00:00
27     2014-10-21 01:00:00
28     2014-10-21 01:00:00
29     2014-10-21 01:00:00
               ...        
4154   2014-10-30 20:00:00
4155   2014-10-30 20:00:00
4156   2014-10-30 21:00:00
4157   2014-10-30 21:00:00
4158   2014-10-30 21:00:00
4159   2014-10-30 21:00:00
4

In [150]:
df['date'] = [d.date() for d in df['new_hour']]
df['time'] = [d.time() for d in df['new_hour']]

In [151]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,click,hour,C1,banner_pos,site_id,site_domain,...,C15,C16,C17,C18,C19,C20,C21,new_hour,date,time
0,0,0,236,13120299559997056165,0,14102100,1005,0,543a539e,c7ca3108,...,320,50,2333,0,39,-1,157,2014-10-21,2014-10-21,00:00:00
1,1,1,259,13447361190641805430,0,14102100,1005,1,17caea14,0dde25ec,...,320,50,1800,3,167,100075,23,2014-10-21,2014-10-21,00:00:00
2,2,2,357,14758321504714974000,0,14102100,1005,0,85f751fd,c4e18dd6,...,320,50,2264,3,427,100000,61,2014-10-21,2014-10-21,00:00:00
3,3,3,455,16246356889796608406,0,14102100,1005,0,5b08c53b,7687a86e,...,300,250,1994,2,39,100084,33,2014-10-21,2014-10-21,00:00:00
4,4,4,540,17569568361412548369,0,14102100,1005,0,543a539e,c7ca3108,...,320,50,2333,0,39,-1,157,2014-10-21,2014-10-21,00:00:00


In [152]:
df = df.drop(columns=['new_hour'])

In [153]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,click,hour,C1,banner_pos,site_id,site_domain,...,C14,C15,C16,C17,C18,C19,C20,C21,date,time
0,0,0,236,13120299559997056165,0,14102100,1005,0,543a539e,c7ca3108,...,20362,320,50,2333,0,39,-1,157,2014-10-21,00:00:00
1,1,1,259,13447361190641805430,0,14102100,1005,1,17caea14,0dde25ec,...,19950,320,50,1800,3,167,100075,23,2014-10-21,00:00:00
2,2,2,357,14758321504714974000,0,14102100,1005,0,85f751fd,c4e18dd6,...,19743,320,50,2264,3,427,100000,61,2014-10-21,00:00:00
3,3,3,455,16246356889796608406,0,14102100,1005,0,5b08c53b,7687a86e,...,17654,300,250,1994,2,39,100084,33,2014-10-21,00:00:00
4,4,4,540,17569568361412548369,0,14102100,1005,0,543a539e,c7ca3108,...,20362,320,50,2333,0,39,-1,157,2014-10-21,00:00:00


In [154]:
df.dtypes



Unnamed: 0           int64
Unnamed: 0.1         int64
Unnamed: 0.1.1       int64
id                  uint64
click                int64
hour                 int64
C1                   int64
banner_pos           int64
site_id             object
site_domain         object
site_category       object
app_id              object
app_domain          object
app_category        object
device_id           object
device_ip           object
device_model        object
device_type          int64
device_conn_type     int64
C14                  int64
C15                  int64
C16                  int64
C17                  int64
C18                  int64
C19                  int64
C20                  int64
C21                  int64
date                object
time                object
dtype: object

In [155]:
#categories = []
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'id', 'click', 'hour',
       'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id',
       'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model',
       'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18',
       'C19', 'C20', 'C21', 'date', 'time'],
      dtype='object')

In [156]:
#converting non numerical columns into numerical columns

categories = ['site_id', 'site_domain','site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'date', 'time']

for category in categories:
  df[category] = df[category].astype('category').cat.codes

In [157]:
df.head()


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,click,hour,C1,banner_pos,site_id,site_domain,...,C14,C15,C16,C17,C18,C19,C20,C21,date,time
0,0,0,236,13120299559997056165,0,14102100,1005,0,113,227,...,20362,320,50,2333,0,39,-1,157,0,0
1,1,1,259,13447361190641805430,0,14102100,1005,1,25,7,...,19950,320,50,1800,3,167,100075,23,0,0
2,2,2,357,14758321504714974000,0,14102100,1005,0,177,225,...,19743,320,50,2264,3,427,100000,61,0,0
3,3,3,455,16246356889796608406,0,14102100,1005,0,122,125,...,17654,300,250,1994,2,39,100084,33,0,0
4,4,4,540,17569568361412548369,0,14102100,1005,0,113,227,...,20362,320,50,2333,0,39,-1,157,0,0


In [158]:


df.head()

features = ['C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'date', 'time']
target = ['click']

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size = 0.2, random_state = 0)




In [159]:
#running random forest algorithm

from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=1000, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)  





/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [160]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 


Mean Absolute Error: 0.2861409796893668
Mean Squared Error: 0.14789977777777777
Root Mean Squared Error: 0.3845774015432755


In [161]:
#running scikit learns recursive feature selection
from sklearn.feature_selection import RFE

rfe = RFE(regressor, n_features_to_select=10, step=1, verbose=0)

rfe = rfe.fit(X_train, y_train)

print(rfe.support_)
print(rfe.ranking_)


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[False False  True  True False False False False False  True  True False
 False  True False False  True False False  True  True  True  True]
[11 12  1  1  5  3  7  9  4  1  1 13 10  1 14  6  1  8  2  1  1  1  1]


In [162]:
rfe_features=[ 'site_id', 'site_domain','device_ip', 'device_model', 'C14', 'C19', 'C20', 'C21', 'date', 'time']

bad_features=['[11]C1', '[12]banner_pos', '[5]site_category', '[4]app_id', '[7]app_domain', 
              '[8]app_category', '[3]device_id', '[13]device_type', '[10]device_conn_type', '[14]C15', '[6]C16', '[9]C17', '[2]C18',]

In [163]:
#rerunning Random Forests with rfe_features
X_train, X_test, y_train, y_test = train_test_split(df[rfe_features], df[target], test_size = 0.2, random_state = 0)
  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [164]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.28365949820788533
Mean Squared Error: 0.14509208243727598
Root Mean Squared Error: 0.3809095462669267
